In [1]:
nn_model_id = 'retrained_sherlock7'

In [2]:
import pandas as pd
import numpy as np

from sherlock.deploy.train_sherlock import train_sherlock
from sherlock.deploy.predict_sherlock import predict_sherlock
from datetime import datetime
from sklearn.metrics import f1_score

### Load training and validation sets

In [3]:
start = datetime.now()
print(f'Started at {start}')

X_train = pd.read_parquet('train.parquet')
y_train = pd.read_parquet('../data/raw/train_labels.parquet').values.flatten()

print(f'Load data (train) process took {datetime.now() - start} seconds.')

Started at 2020-12-31 10:01:09.253953
Load data (train) process took 0:00:03.333291 seconds.


In [4]:
print('Distinct types for columns in the Dataframe (should be all float32):')
print(set(X_train.dtypes))

Distinct types for columns in the Dataframe (should be all float32):
{dtype('float32')}


In [5]:
start = datetime.now()
print(f'Started at {start}')

X_validation = pd.read_parquet('validation.parquet')
y_validation = pd.read_parquet('../data/raw/val_labels.parquet').values.flatten()

print(f'Load data (validation) process took {datetime.now() - start} seconds.')

Started at 2020-12-31 10:01:12.598487
Load data (validation) process took 0:00:01.320111 seconds.


### Re-train Sherlock

In [6]:
start = datetime.now()
print(f'Started at {start}')

train_sherlock(X_train, y_train, X_validation, y_validation, nn_id=nn_model_id);

print('Trained and saved new model.')
print(f'Finished at {datetime.now()}, took {datetime.now() - start} seconds')

Started at 2020-12-31 10:01:13.923603


W1231 10:01:14.128250 4476407232 deprecation.py:506] From /Users/lowecg/source/private-github/sherlock-project/venv/lib/python3.7/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1231 10:01:14.129240 4476407232 deprecation.py:506] From /Users/lowecg/source/private-github/sherlock-project/venv/lib/python3.7/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1231 10:01:14.132912 4476407232 deprecation.py:506] From /Users/lowecg/source/private-github/sherlock-project/venv/lib/python3.7/site-pa

Successfully loaded and compiled model, now fitting model on data.
Train on 412059 samples, validate on 137353 samples
Epoch 1/100
412059/412059 [==============================] - 65s 158us/sample - loss: 1.6124 - categorical_accuracy: 0.6976 - val_loss: 1.0390 - val_categorical_accuracy: 0.8285
Epoch 2/100
412059/412059 [==============================] - 64s 156us/sample - loss: 0.9660 - categorical_accuracy: 0.8359 - val_loss: 0.9434 - val_categorical_accuracy: 0.8498
Epoch 3/100
412059/412059 [==============================] - 63s 153us/sample - loss: 0.8520 - categorical_accuracy: 0.8591 - val_loss: 0.8939 - val_categorical_accuracy: 0.8570
Epoch 4/100
412059/412059 [==============================] - 62s 151us/sample - loss: 0.7871 - categorical_accuracy: 0.8708 - val_loss: 0.8584 - val_categorical_accuracy: 0.8626
Epoch 5/100
412059/412059 [==============================] - 63s 152us/sample - loss: 0.7389 - categorical_accuracy: 0.8788 - val_loss: 0.8305 - val_categorical_accuracy

### Load test set

In [7]:
start = datetime.now()
print(f'Started at {start}')

X_test = pd.read_parquet('test.parquet')
y_test = pd.read_parquet('../data/raw/test_labels.parquet').values.flatten()

print('Trained and saved new model.')
print(f'Finished at {datetime.now()}, took {datetime.now() - start} seconds')

Started at 2020-12-31 10:43:11.718561
Trained and saved new model.
Finished at 2020-12-31 10:43:13.631580, took 0:00:01.913031 seconds


### Make prediction

In [8]:
predicted_labels = predict_sherlock(X_test, nn_id=nn_model_id)

In [9]:
print(f'prediction count {len(predicted_labels)}, type = {type(predicted_labels)}')

size=len(y_test)

# Should be fully deterministic too.
f1_score(y_test[:size], predicted_labels[:size], average="weighted")

prediction count 137353, type = <class 'numpy.ndarray'>


0.8940497565462086

In [10]:
# Best results
# 0.8946297512938394

# LAST
# 0.8661276466591463
# 0.8765360861865846
# 0.8777825108666634
# 0.8709458033528152

## Review errors

In [11]:
from collections import Counter

size = len(y_test)
mismatches = list()

for idx, k1 in enumerate(y_test[:size]):
    k2 = predicted_labels[idx]

    if k1 != k2:
        mismatches.append(k1)
#        if k1 in ('brand'):
#        print(f'[{idx}] expected "{k1}" but predicted "{k2}"')
        
f1 = f1_score(y_test[:size], predicted_labels[:size], average="weighted")
print(f'Total mismatches: {len(mismatches)} (F1 score: {f1})')

data = Counter(mismatches)
data.most_common()   # Returns all unique items and their counts

Total mismatches: 14614 (F1 score: 0.8940497565462086)


[('name', 764),
 ('position', 557),
 ('location', 501),
 ('region', 488),
 ('rank', 476),
 ('description', 438),
 ('artist', 376),
 ('team', 371),
 ('area', 359),
 ('album', 357),
 ('notes', 350),
 ('type', 349),
 ('product', 346),
 ('company', 338),
 ('day', 318),
 ('ranking', 316),
 ('category', 314),
 ('team Name', 295),
 ('city', 291),
 ('class', 269),
 ('code', 252),
 ('order', 252),
 ('gender', 244),
 ('country', 236),
 ('sex', 222),
 ('owner', 214),
 ('status', 212),
 ('person', 211),
 ('result', 189),
 ('plays', 185),
 ('address', 182),
 ('duration', 175),
 ('year', 175),
 ('credit', 170),
 ('sales', 170),
 ('service', 167),
 ('brand', 165),
 ('manufacturer', 164),
 ('weight', 162),
 ('range', 161),
 ('age', 160),
 ('component', 155),
 ('state', 153),
 ('origin', 153),
 ('format', 147),
 ('club', 135),
 ('county', 118),
 ('director', 117),
 ('capacity', 105),
 ('publisher', 98),
 ('affiliation', 89),
 ('command', 89),
 ('classification', 85),
 ('family', 81),
 ('nationality', 8

In [12]:
test_samples = pd.read_parquet('../data/raw/test_values.parquet')

In [13]:
from ast import literal_eval

idx = 541
original = test_samples.iloc[idx]
converted = original.apply(literal_eval).to_list()

print(f'Predicted "{predicted_labels[idx]}", actual label "{y_test[idx]}". Actual values:\n{converted}')

Predicted "age", actual label "position". Actual values:
[[2, 4]]
